In [ ]:
!pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

## Load the data

In [ ]:
# Abalone dataset https://archive.ics.uci.edu/ml/datasets/abalone
df_abalone_train = pd.read_csv(
    "https://storage.googleapis.com/download.tensorflow.org/data/abalone_train.csv", header=None, 
    names=["Length", "Diameter", "Height", "Whole weight", "Shucked weight",
           "Viscera weight", "Shell weight", "Age"])

df_abalone_train.head()

In [ ]:
df_abalone_train.shape

The nominal task for this dataset is to predict the age from the other measurements, so separate the features and labels for training:

In [ ]:
abalone_features = df_abalone_train.copy()
abalone_labels = abalone_features.pop('Age')

In [ ]:
# Pack the features into a single NumPy array
abalone_features = np.array(abalone_features)

In [ ]:
abalone_features.shape

## Modeling

The basic building block of a neural network is the *layer*. Layers extract representations from the data fed into them. 

Most of deep learning consists of chaining together simple layers. Most layers, such as `tf.keras.layers.Dense`, have parameters that are learned during training.

In [ ]:
# Basic with no preprocessing
abalone_model = tf.keras.Sequential([
  layers.Dense(64),
  layers.Dense(1)
])

Before the model is ready for training, it needs a few more settings. These are added during the model's compile step:
* Loss function —This measures how accurate the model is during training. You want to minimize this function to "steer" the model in the right direction.
* Optimizer —This is how the model is updated based on the data it sees and its loss function.
* Metrics —Used to monitor the training and testing steps.

In [ ]:
abalone_model.compile(loss=tf.losses.MeanSquaredError(),
                      optimizer=tf.optimizers.Adam())

In [ ]:
abalone_model.build(input_shape=abalone_features.shape)

In [ ]:
abalone_features.shape

In [ ]:
abalone_model.summary()

In [ ]:
abalone_model.fit(abalone_features, abalone_labels, epochs=10)

## Preprocessing

It's good practice to normalize the inputs to your model. The `experimental.preprocessing` layers provide a convenient way to build this normalization into your model.

In [ ]:
normalize = preprocessing.Normalization()

**Note: Only use your training data to .adapt() preprocessing layers. Do not use your validation or test data.**

In [ ]:
normalize.adapt(abalone_features)

In [ ]:
# use the normalization layer in the model
norm_abalone_model = tf.keras.Sequential([
  normalize,
  layers.Dense(64),
  layers.Dense(1)
])

norm_abalone_model.compile(loss = tf.losses.MeanSquaredError(),
                           optimizer = tf.optimizers.Adam())

norm_abalone_model.fit(abalone_features, abalone_labels, epochs=10)

We use a loss function to determine how far the predicted values deviate from the actual values in the training data. ... We change the model weights to make the loss minimum, and that is what training is all about

In [ ]:
norm_abalone_model.evaluate(abalone_features, abalone_labels)

In [ ]:
abalone_features[10], abalone_labels[10]

In [ ]:
norm_abalone_model.predict(abalone_features[10][np.newaxis,:])

## Classification Example

In [ ]:
mnist = tf.keras.datasets.mnist

In [ ]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

The first layer in this network, tf.keras.layers.Flatten, transforms the format of the images from a two-dimensional array (of 28 by 28 pixels) to a one-dimensional array (of 28 * 28 = 784 pixels). Think of this layer as unstacking rows of pixels in the image and lining them up. This layer has no parameters to learn; it only reformats the data.

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

training_history = model.fit(x_train, 
                             y_train, 
                             epochs=10,
                             validation_split=0.2)

In [ ]:
model.summary()

In [ ]:
# Evaluate returns the loss value and metrics values for the model.
model.evaluate(x_test, y_test)

You can learn a lot about neural networks and deep learning models by observing their performance over time during training.

In [ ]:
training_history.history.keys()

In [ ]:
# summarize history for accuracy
plt.plot(training_history.history['accuracy'])
plt.plot(training_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(training_history.history['loss'])
plt.plot(training_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()